In [1]:
!pip install -q tensorflow
!pip install -q numpy
!pip install -q pandas
!pip install -q matplotlib
!pip install spacy


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import random
import spacy

from spacy.training import Example

%matplotlib inline


In [3]:
df = pd.read_csv("data/train.csv")


In [4]:
df.head()


,ID,URL,Product_Name,Full_Content,Section_1,Section_2,Section_3,Section_4,Section_5,Section_6
0,5C706E5A9E28B681F969CE8D6DC55607,https://www.ema.europa.eu/documents/product-in...,Pergoveris,1\n\nANNEX I\n\nSUMMARY OF PRODUCT CHARACTERIS...,{'Title': '1. what pergoveris is and what it i...,{'Title': '2. what you need to know before you...,"{'Title': '3. how to use pergoveris', 'Section...","{'Title': '4. possible side effects', 'Section...","{'Title': '5. how to store pergoveris', 'Secti...",{'Title': '6. contents of the pack and other i...
1,17F9F8809EB0FC0F70E195786F80368D,https://www.ema.europa.eu/documents/product-in...,Refludan,1 \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n...,{'Title': '1. what refludan is and what it is ...,"{'Title': '2. before you use refludan', 'Secti...","{'Title': '3. how to use refludan', 'Section_C...","{'Title': '4. possible side effects', 'Section...","{'Title': '5 how to store refludan', 'Section_...","{'Title': '6. further information', 'Section_C..."
2,4263665B0FB8C7F2A0039D03A3678097,https://www.ema.europa.eu/documents/product-in...,Onsenal,1 \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,{'Title': '1. what onsenal is and what it is u...,"{'Title': '2. before you take onsenal', 'Secti...","{'Title': '3. how to take onsenal', 'Section_C...","{'Title': '4. possible side effects', 'Section...","{'Title': '5. how to store onsenal', 'Section_...","{'Title': '6. further information', 'Section_C..."
3,0B6B1FB1EF2BF8BE2D880507FBFFF23E,https://www.ema.europa.eu/documents/product-in...,Nerlynx,1 \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,{'Title': '1. what nerlynx is and what it is u...,{'Title': '2. what you need to know before you...,"{'Title': '3. how to take nerlynx', 'Section_C...","{'Title': '4. possible side effects', 'Section...","{'Title': '5. how to store nerlynx', 'Section_...",{'Title': '6. contents of the pack and other i...
4,843352215D2AEF5460300234FD08912B,https://www.ema.europa.eu/documents/product-in...,Oncaspar,1 \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,{'Title': '1. what oncaspar is and what it is ...,{'Title': '2. what you need to know before you...,"{'Title': '3. how oncaspar is given', 'Section...","{'Title': '4. possible side effects', 'Section...","{'Title': '5. how to store oncaspar', 'Section...",{'Title': '6. contents of the pack and other i...


In [5]:
d = ast.literal_eval(df.Section_1[68])


In [6]:
d


{'Title': '1. what leganto is and what it is used for',
 'Section_Content': "what leganto is leganto contains the active substance rotigotine. it belongs to a group of medicines called 'dopamine agonists'. dopamine is a messenger in the brain which is important for movement. what leganto is used for leganto is used in adults to treat the signs and symptoms of: restless legs syndrome (rls) this can be associated with discomfort in your legs or arms, urges to move around, sleep disturbance and feeling tired or sleepy during the day. these symptoms are either reduced or their duration is shortened with leganto treatment.",
 'Entity_Recognition': [{'Text': 'leganto',
   'Type': 'PRODUCT_NAME',
   'BeginOffset': 0,
   'EndOffset': 0},
  {'Text': 'the active substance rotigotine',
   'Type': 'TREATMENT',
   'BeginOffset': 33,
   'EndOffset': 64},
  {'Text': "'dopamine agonists",
   'Type': 'TREATMENT',
   'BeginOffset': 108,
   'EndOffset': 126},
  {'Id': 6,
   'BeginOffset': 129,
   'EndOff

In [7]:
%%time
labels = []
TRAIN_DATA = []

for sec_i in range(1, 2):
    print(f'Section_{sec_i}')
    for j in range(len(df)):
        data = ast.literal_eval(df[f'Section_{sec_i}'][j])
        text = data["Section_Content"]
        new_anno = []
        indexes = set()
        if data['Entity_Recognition'] is None:
            continue
        for entity in data['Entity_Recognition']:
            if 'Type' not in entity.keys():
                continue
            start = entity['BeginOffset']
            end = entity['EndOffset']
            label = entity['Type']
            if label not in labels:
                labels.append(label)
            start_to_end = list(range(start, end + 1))
            if (start in indexes) or (end in indexes) or (any([i in indexes for i in start_to_end])) or (start == end):
                continue
            new_anno.append((start, end, label))
            indexes.update(start_to_end)
        TRAIN_DATA.append((text, {"entities": new_anno}))


Section_1
CPU times: user 921 ms, sys: 13.5 ms, total: 935 ms
Wall time: 935 ms


In [9]:
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")


In [10]:
for ent in labels:
    ner.add_label(ent)


In [11]:
nlp.make_doc(TRAIN_DATA[67][0])


what leganto is leganto contains the active substance rotigotine. it belongs to a group of medicines called 'dopamine agonists'. dopamine is a messenger in the brain which is important for movement. what leganto is used for leganto is used in adults to treat the signs and symptoms of: restless legs syndrome (rls) this can be associated with discomfort in your legs or arms, urges to move around, sleep disturbance and feeling tired or sleepy during the day. these symptoms are either reduced or their duration is shortened with leganto treatment.

In [12]:
TRAIN_DATA[449]


('granpidam contains the active substance sildenafil which belongs to a group of medicines called phosphodiesterase type 5 (pde5) inhibitors. granpidam brings down blood pressure in the lungs by widening the blood vessels in the lungs. granpidam is used to treat adults and children and adolescents from 1 to 17 years old with high blood pressure in the blood vessels in the lungs (pulmonary arterial hypertension).',
 {'entities': [(0, 9, 'GENERIC_NAME'),
   (19, 50, 'TREATMENT'),
   (68, 88, 'TREATMENT'),
   (96, 138, 'TREATMENT_NAME'),
   (162, 176, 'TEST'),
   (184, 189, 'SYSTEM_ORGAN_SITE'),
   (193, 232, 'PROBLEM'),
   (302, 303, 'NUMBER'),
   (307, 309, 'NUMBER'),
   (325, 378, 'PROBLEM'),
   (380, 411, 'DX_NAME')]})

In [13]:
import warnings
warnings.filterwarnings('ignore')


In [16]:
%%time

from tqdm import tqdm_notebook

epochs = 25

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):

    optimizer = nlp.begin_training()

    for i in tqdm_notebook(range(epochs)):
        random.shuffle(TRAIN_DATA)
        for text, annotation in tqdm_notebook(TRAIN_DATA):
            if text is None:
                continue
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], sgd=optimizer)


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

...

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

  0%|          | 0/1055 [00:00<?, ?it/s]

CPU times: user 18min 53s, sys: 1min 34s, total: 20min 27s
Wall time: 18min 29s


In [17]:
test = pd.read_csv('data/test.csv')

In [18]:
test_d = ast.literal_eval(test.Section_1[0])

In [19]:
test_doc = nlp(test_d['Section_Content'])

In [ ]:
from spacy import displacy
displacy.serve(test_doc, style="ent", port=4444)


Using the 'ent' visualizer
Serving on http://0.0.0.0:4444 ...



In [ ]:
ner.to_disk("navi_ner")